<a href="https://colab.research.google.com/github/TBKHori/Music-Recon13/blob/main/Test_RL_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stable_baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 8.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pytest

In [3]:
from stable_baselines3 import A2C, DDPG, DQN, PPO, SAC, TD3
from stable_baselines3.common.envs import IdentityEnv, IdentityEnvBox, IdentityEnvMultiBinary, IdentityEnvMultiDiscrete
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv

In [11]:
DIM = 4

In [12]:
@pytest.mark.parametrize("model_class", [A2C, PPO, SAC, DDPG, TD3])
def test_continuous(model_class):
    env = IdentityEnvBox(eps=0.5)

    n_steps = {A2C: 2000, PPO: 2000, SAC: 400, TD3: 400, DDPG: 400}[model_class]

    kwargs = dict(policy_kwargs=dict(net_arch=[64, 64]), seed=0, gamma=0.95)

    if model_class in [TD3]:
        n_actions = 1
        action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
        kwargs["action_noise"] = action_noise
    elif model_class in [A2C]:
        kwargs["policy_kwargs"]["log_std_init"] = -0.5
    elif model_class == PPO:
        kwargs = dict(n_steps=512, n_epochs=5)


In [13]:
def test_discrete(model_class, env):
    env_ = DummyVecEnv([lambda: env])
    kwargs = {}
    n_steps = 2500
    if model_class == DQN:
        kwargs = dict(learning_starts=0)
        # DQN only support discrete actions
        if isinstance(env, (IdentityEnvMultiDiscrete, IdentityEnvMultiBinary)):
            return

In [15]:
# model_class is the class of the RL model to be used, e.g., PPO, A2C, etc.
# "MlpPolicy" specifies the policy architecture, which is Multi-Layer Perceptron (MLP) in this case.
# env_ is the environment in which the RL model will be trained.
# gamma is the discount factor used in the RL algorithm.
# seed is the random seed for reproducibility.
# **kwargs represents any additional hyperparameters that can be passed to the model_class.
model = model_class("MlpPolicy", env_, gamma=0.4, seed=3, **kwargs).learn(n_steps)

# After training, evaluate the trained RL model on the environment env_.
# n_eval_episodes specifies the number of episodes for evaluation.
# reward_threshold is the minimum reward required for success.
# warn=False suppresses warning messages during evaluation.
evaluate_policy(model, env_, n_eval_episodes=20, reward_threshold=90, warn=False)

# Reset the environment and get the initial observation (state).
obs, _ = env.reset()

# Assert the shape of the model's prediction is the same as the shape of the observation.
# This is a basic check to ensure the model can predict actions based on observations.
assert np.shape(model.predict(obs)[0]) == np.shape(obs)


In [15]:
@pytest.mark.parametrize("model_class", [A2C, PPO, SAC, DDPG, TD3])
def test_continuous(model_class):
    env = IdentityEnvBox(eps=0.5)

    n_steps = {A2C: 2000, PPO: 2000, SAC: 400, TD3: 400, DDPG: 400}[model_class]

    kwargs = dict(policy_kwargs=dict(net_arch=[64, 64]), seed=0, gamma=0.95)

    if model_class in [TD3]:
        n_actions = 1
        action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
        kwargs["action_noise"] = action_noise
    elif model_class in [A2C]:
        kwargs["policy_kwargs"]["log_std_init"] = -0.5
    elif model_class == PPO:
        kwargs = dict(n_steps=512, n_epochs=5)

In [15]:
# model_class is the class of the RL model to be used, e.g., PPO, A2C, etc.
# "MlpPolicy" specifies the policy architecture, which is Multi-Layer Perceptron (MLP) in this case.
# env is the environment in which the RL model will be trained and evaluated.
# learning_rate is the learning rate used by the optimizer during training.
# **kwargs represents any additional hyperparameters that can be passed to the model_class.
model = model_class("MlpPolicy", env, learning_rate=1e-3, **kwargs).learn(n_steps)

# After training, evaluate the trained RL model on the same environment.
# n_eval_episodes specifies the number of episodes for evaluation.
# reward_threshold is the minimum reward required for success.
# warn=False suppresses warning messages during evaluation.
evaluate_policy(model, env, n_eval_episodes=20, reward_threshold=90, warn=False)
